# How to play with Memgraph in a Jupyter Notebook? 

Through this short tutorial, you will learn how to install Memgraph, connect to it from a Jupyter Notebook and perform data analysis using graph algorithms.

## 1. Prerequisites

For this tutorial, you will need to install:
- [Jupyter](https://jupyter.org/install)
- [Docker](https://docs.docker.com/get-docker/)

Docker is used because Memgraph is a native Linux application and cannot be installed on Windows and macOS.

## 2. Installation using Docker

After you install Docker, you can set up Memgraph by running:

```
docker run -it -p 7687:7687 -p 3000:3000 memgraph/memgraph-platform
```

This command will start the download and after it finishes, run the Memgraph container.

## 3. Connecting to Memgraph with GQLAlchemy

We will be using the **GQLAlchemy** object graph mapper (OGM) to connect to Memgraph and execute **Cypher** queries easily. GQLAlchemy also serves as a Python driver/client for Memgraph. You can install it using:

```
pip install gqlalchemy
```

> **Hint**: You may need to install [CMake](https://cmake.org/download/) before installing GQLAlchemy.

Maybe you got confused when I mentioned Cypher. You can think of Cypher as SQL for graph databases. It contains many of the same language constructs like `CREATE`, `UPDATE`, `DELETE`... and it's used to query the database.

In [1]:
from gqlalchemy import Memgraph

In [2]:
memgraph = Memgraph("127.0.0.1", 7687)

Let's make sure that Memgraph is empty before we start with anything else.

In [3]:
memgraph.drop_database()

## 4. Importing data from CSV files

You will need to download [this file](https://github.com/g-despot/jupyter-memgraph-tutorials/blob/main/karate-club.csv) which contains a simple dataset called Zachary's karate club. To import it into Memgraph, we will first need to copy it to the Docker container where Memgraph is running. 
Find the `CONTAINER_ID` by running:

```
docker ps
```

Copy the file with the following command (don't forget to replace `CONTAINER_ID`):

```
docker cp karate-club.csv CONTAINER_ID:karate-club.csv
```

Now, we can execute the Cypher command `LOAD CSV`, which is used for loading data from CSV files:

In [4]:
memgraph.execute("""
    LOAD CSV FROM "/karate-club.csv" NO HEADER AS row
    MERGE (p1:Person {id: row[0]})
    MERGE (p2:Person {id: row[1]})
    MERGE (p1)-[:FRIENDS_WITH]->(p2);
""")

## 5. Querying the database and retrieving results

Let's make sure that our data was imported correctly by retrieving it:

In [5]:
results = memgraph.execute_and_fetch("""
    MATCH (p1:Person)
    RETURN p1
    ORDER BY ToInteger(p1.id) ASC;
""")

In [6]:
for result in results:
    print(result['p1'])

<Node id=1730 labels={'Person'} properties={'id': '1'}>
<Node id=1731 labels={'Person'} properties={'id': '2'}>
<Node id=1732 labels={'Person'} properties={'id': '3'}>
<Node id=1733 labels={'Person'} properties={'id': '4'}>
<Node id=1734 labels={'Person'} properties={'id': '5'}>
<Node id=1735 labels={'Person'} properties={'id': '6'}>
<Node id=1736 labels={'Person'} properties={'id': '7'}>
<Node id=1737 labels={'Person'} properties={'id': '8'}>
<Node id=1738 labels={'Person'} properties={'id': '9'}>
<Node id=1739 labels={'Person'} properties={'id': '10'}>
<Node id=1740 labels={'Person'} properties={'id': '11'}>
<Node id=1741 labels={'Person'} properties={'id': '12'}>
<Node id=1742 labels={'Person'} properties={'id': '13'}>
<Node id=1743 labels={'Person'} properties={'id': '14'}>
<Node id=1758 labels={'Person'} properties={'id': '15'}>
<Node id=1759 labels={'Person'} properties={'id': '16'}>
<Node id=1744 labels={'Person'} properties={'id': '17'}>
<Node id=1745 labels={'Person'} properti

## 6. Calculating PageRank

Now, let's do something clever with our graph. 
For example, calculating PageRank for each node:

In [7]:
results = memgraph.execute_and_fetch("""
    CALL pagerank.get()
    YIELD node, rank
    RETURN node, rank;
""")

In [8]:
for result in results:
    print("The PageRank of node ", result['node'].properties['id'], ": ", result['rank'])

The PageRank of node  1 :  0.015060494762879958
The PageRank of node  2 :  0.015860583547157955
The PageRank of node  3 :  0.01754577054904349
The PageRank of node  4 :  0.01941000866987936
The PageRank of node  5 :  0.015860583547157955
The PageRank of node  6 :  0.015860583547157955
The PageRank of node  7 :  0.027095163559728172
The PageRank of node  8 :  0.02490951112634518
The PageRank of node  9 :  0.017724821667993825
The PageRank of node  10 :  0.016924732883715828
The PageRank of node  11 :  0.027095163559728172
The PageRank of node  12 :  0.015860583547157955
The PageRank of node  13 :  0.021360086003623775
The PageRank of node  14 :  0.02490951112634518
The PageRank of node  17 :  0.04258521579367699
The PageRank of node  18 :  0.01754577054904349
The PageRank of node  20 :  0.01754577054904349
The PageRank of node  22 :  0.01754577054904349
The PageRank of node  24 :  0.015060494762879958
The PageRank of node  26 :  0.02188791905538554
The PageRank of node  25 :  0.01506049

## 7. Calculating betweenness centrality

You probably get the gist now: 

In [9]:
results = memgraph.execute_and_fetch("""
    CALL betweenness_centrality.get()
    YIELD node, betweenness_centrality
    RETURN node, betweenness_centrality;
""")

In [10]:
for result in results:
    print("The betweenness centrality of node ", result['node'].properties['id'], ": ", result['betweenness_centrality'])

The betweenness centrality of node  1 :  0.0
The betweenness centrality of node  2 :  0.0004734848484848485
The betweenness centrality of node  3 :  0.008364898989898988
The betweenness centrality of node  4 :  0.001893939393939394
The betweenness centrality of node  5 :  0.0
The betweenness centrality of node  6 :  0.0004734848484848485
The betweenness centrality of node  7 :  0.0014204545454545455
The betweenness centrality of node  8 :  0.0
The betweenness centrality of node  9 :  0.002130681818181818
The betweenness centrality of node  10 :  0.00015782828282828281
The betweenness centrality of node  11 :  0.0
The betweenness centrality of node  12 :  0.0
The betweenness centrality of node  13 :  0.0
The betweenness centrality of node  14 :  0.0016571969696969697
The betweenness centrality of node  17 :  0.0
The betweenness centrality of node  18 :  0.0
The betweenness centrality of node  20 :  0.0005523989898989899
The betweenness centrality of node  22 :  0.0
The betweenness centr

Visit the [Memgraph MAGE](https://github.com/memgraph/mage) graph library, which contains many other algorithms. You can also implement and submit your own algorithms and utility procedures.

## What's next?

Now it's time for you to use Memgraph on a graph problem!

You can always check out [Memgraph Playground](https://playground.memgraph.com/) for some cool use cases and examples.
Definitely don't forget to register for the **[Memgraph App Challenge](https://memgraph.com/memgraph-app-challenge)** and check out **[what else you can work on](https://memgraph.com/memgraph-app-challenge#options)** if graph analysis with Jupyter is not your cup of tea.